# Machine learning project - Mikołaj Mazur

In [1]:
from datetime import datetime
import pandas as pd

### Dataset sources

## Data Exploration

In [2]:
# Chicago Bike Sharing Data

bike_data = pd.read_csv('chicago_bike_sharing_2021.csv')
print(bike_data.head())
print(bike_data.info())

   Unnamed: 0           ride_id  rideable_type           started_at  \
0           0  E19E6F1B8D4C42ED  electric_bike  2021-01-23 16:14:19   
1           1  DC88F20C2C55F27F  electric_bike  2021-01-27 18:43:08   
2           2  EC45C94683FE3F27  electric_bike  2021-01-21 22:35:54   
3           3  4FA453A75AE377DB  electric_bike  2021-01-07 13:31:13   
4           4  BE5E8EB4E7263A0B  electric_bike  2021-01-23 02:24:02   

              ended_at          start_station_name start_station_id  \
0  2021-01-23 16:24:44  California Ave & Cortez St            17660   
1  2021-01-27 18:47:12  California Ave & Cortez St            17660   
2  2021-01-21 22:37:14  California Ave & Cortez St            17660   
3  2021-01-07 13:42:55  California Ave & Cortez St            17660   
4  2021-01-23 02:24:45  California Ave & Cortez St            17660   

  end_station_name end_station_id  start_lat  start_lng  end_lat  end_lng  \
0              NaN            NaN  41.900341 -87.696743    41.89   -8

In [3]:
# Weather data

weather_data = pd.read_csv('weather_data.csv')
print(weather_data.head())
print(weather_data.info())

                                                                                                                     CHICAGO MIDWAY AIRPORT 3 SW, IL US (USC00111577)
Date       TAVG (Degrees Fahrenheit) TMAX (Degrees Fahrenheit) TMIN (Degrees Fahrenheit) PRCP (Inches) SNOW (Inches)                                    SNWD (Inches)
1928-02-29 NaN                       39                        27                        0.00          0.0                                                        0.0
1928-03-01 NaN                       36                        20                        0.00          0.0                                                        0.0
1928-03-02 NaN                       42                        25                        0.00          0.0                                                        0.0
1928-03-03 NaN                       37                        15                        0.00          0.0                                                        0.0
<cla

## Preparation of data

### Remove irrelevant columns

In [4]:
bike_data = bike_data[['started_at']]
bike_data

,started_at
0,2021-01-23 16:14:19
1,2021-01-27 18:43:08
2,2021-01-21 22:35:54
3,2021-01-07 13:31:13
4,2021-01-23 02:24:02
...,...
5595058,2021-12-12 13:36:55
5595059,2021-12-06 19:37:50
5595060,2021-12-02 08:57:04
5595061,2021-12-13 09:00:26


In [5]:
# format date

bike_data = bike_data['started_at'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')
)

In [6]:
bike_data = bike_data.to_frame()

### Group by data

In [7]:
bike_data_count = bike_data.groupby('started_at')['started_at'].count().to_frame()

In [8]:
bike_data_count

,started_at
started_at,
2021-01-01,1068
2021-01-02,2275
2021-01-03,2383
2021-01-04,3209
2021-01-05,3845
...,...
2021-12-27,4903
2021-12-28,2830
2021-12-29,5370


### NaN check

In [9]:
bike_data_count.isna().sum()

started_at    0
dtype: int64